In [ ]:
import pandas as pd
import plotly.express as px
import base64
from IPython.display import HTML

pd.options.plotting.backend = "plotly"

# 2 Jornalista quer automatizar a exportação das estatísticas
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [ ]:
col_eleitorado = [
    "ANO_ELEICAO",
    "SG_UF",
    "NM_MUNICIPIO",
    "DS_ESTADO_CIVIL",
    "CD_FAIXA_ETARIA",
    "DS_FAIXA_ETARIA",
    "CD_GRAU_ESCOLARIDADE",
    "DS_GRAU_ESCOLARIDADE",
    "QT_ELEITORES_PERFIL",
    "QT_ELEITORES_INC_NM_SOCIAL",
]

arquivo_eleitorado_2020 = "./data/perfil_eleitorado_2020.csv"
arquivo_eleitorado_2016 = "./data/perfil_eleitorado_2016.csv"


df_eleitores_2020 = pd.read_csv(
    arquivo_eleitorado_2020,
    usecols=col_eleitorado,
    sep=";",
    encoding="iso-8859-1",
    error_bad_lines=False,
)

df_eleitores_2016 = pd.read_csv(
    arquivo_eleitorado_2016,
    usecols=col_eleitorado,
    sep=";",
    encoding="iso-8859-1",
    error_bad_lines=False,
)


In [ ]:
# 01 Jornalista quer saber a classificação do eleitorado a partir do estado civil, escolaridade e faixa etária, além da quantidade de eleitores que utilizam nome social

In [ ]:
classificacao_estado_civil = (
    df_eleitores_2020["DS_ESTADO_CIVIL"]
    .value_counts()
    .rename_axis("estado_civil")
    .reset_index(name="counts")
)

classificacao_estado_civil
fig = px.bar(
    classificacao_estado_civil,
    x="estado_civil",
    y="counts",
    title="Classificacao por estado civil",
)
fig.show()

create_download_link(classificacao_estado_civil, "Classificacao por estado civil", "classificacao_estado_civil.csv")

In [ ]:
classificacao_grau_escolaridade = (
    df_eleitores_2020["DS_GRAU_ESCOLARIDADE"]
    .value_counts()
    .rename_axis("grau_escolaridade")
    .reset_index(name="counts")
)

classificacao_grau_escolaridade
fig = px.bar(
    classificacao_grau_escolaridade,
    x="grau_escolaridade",
    y="counts",
    title="Classificacao por grau de escolaridade",
)
fig.show()

create_download_link(classificacao_grau_escolaridade, "Classificacao por grau de escolaridade", "classificacao_grau_escolaridade.csv")

In [ ]:
classificacao_faixa_etaria = (
    df_eleitores_2020["DS_FAIXA_ETARIA"]
    .value_counts()
    .rename_axis("faixa_etaria")
    .reset_index(name="counts")
)

classificacao_faixa_etaria
fig = px.bar(
    classificacao_faixa_etaria,
    x="faixa_etaria",
    y="counts",
    title="Classificacao por faixa etaria",
)
fig.show()

create_download_link(classificacao_faixa_etaria, "Classificacao por faixa etaria", "classificacao_faixa_etaria.csv")

In [ ]:
# 02 Jornalista quer relacionar as características do eleitorado de acordo com o representante eleito em determinada região

In [ ]:
arquivo_candidatos_2020 = './data/consulta_cand_2020_SP.csv'

df_candidatos_2020 = pd.read_csv(
    arquivo_candidatos_2020,
    encoding="iso-8859-1",
    error_bad_lines=False,
    sep=";"
)

candidato_sjc = df_candidatos_2020.query('DS_SIT_TOT_TURNO == "ELEITO" and DS_CARGO == "PREFEITO" and NM_UE == "SÃO JOSÉ DOS CAMPOS"')[
    ['NM_CANDIDATO', 'NM_UE', 'DS_GRAU_INSTRUCAO', 'DS_ESTADO_CIVIL']]

nome_prefeito = candidato_sjc.NM_CANDIDATO.values[0]
grau_instrucao = candidato_sjc.DS_GRAU_INSTRUCAO.values[0]
estado_civil = candidato_sjc.DS_ESTADO_CIVIL.values[0]

classificado_grau_escolaridade = df_eleitores_2020 \
    .query('NM_MUNICIPIO == "SÃO JOSÉ DOS CAMPOS"')['DS_GRAU_ESCOLARIDADE'] \
    .value_counts() \
    .rename_axis("grau_escolaridade") \
    .reset_index(name="counts")

fig = px.bar(
    classificado_grau_escolaridade,
    x="grau_escolaridade",
    y="counts",
    title=f"Grau escolaridade eleitorado de SÃO JOSÉ DOS CAMPOS em 2020 - {nome_prefeito}, {grau_instrucao}",
)

fig.show()

create_download_link(classificado_grau_escolaridade, f"Grau escolaridade eleitorado de SÃO JOSÉ DOS CAMPOS em 2020 - {nome_prefeito}, {grau_instrucao}", "classificado_grau_escolaridade_2020.csv")

In [ ]:
classificado_estado_civil = df_eleitores_2020 \
    .query('NM_MUNICIPIO == "SÃO JOSÉ DOS CAMPOS"')['DS_ESTADO_CIVIL'] \
    .value_counts() \
    .rename_axis("estado_civil") \
    .reset_index(name="counts")

fig = px.bar(
    classificado_estado_civil,
    x="estado_civil",
    y="counts",
    title=f"Estado civil eleitorado de SÃO JOSÉ DOS CAMPOS em 2020 - {nome_prefeito}, {estado_civil}",
)

fig.show()

create_download_link(classificado_estado_civil, f"Estado civil eleitorado de SÃO JOSÉ DOS CAMPOS em 2020 - {nome_prefeito}, {estado_civil}", "classificado_estado_civil_2020.csv")

In [ ]:
# 03 Jornalista quer saber a renda média do eleitorado de determinada região

In [ ]:
arquivo_salarios = "./data/Cadastro_Central_de_Empresas_2.csv"
cols_salario = ["Nome", "Localidade", "2019"]
cidades_listadas = [
    "São José dos Campos",
    "São Paulo",
    "Jacareí",
    "São Bento do Sapucaí",
]
df_salarios = pd.read_csv(
    arquivo_salarios, usecols=cols_salario, sep=",", error_bad_lines=False
)
df_salarios_listados = (
    dfr.query('Nome == "Salário médio mensal"')
    .query(f"Localidade in {cidades_listadas}")[["Localidade", "2019"]]
    .rename(columns={"Localidade": "Cidade", "2019": "Salario"})
    .sort_values(by="Salario", ascending=False)
)

fig = px.bar(
    df_salarios_listados, x="Cidade", y="Salario", title="Media salarial por cidade"
)
fig.show()

# create_download_link(df_salarios_listados, "Media salarial por cidade", "media_salarial_cidade.csv")

In [ ]:
# 04 Jornalista quer saber, daquela região, a cidade com mais eleitores jovem e mais eleitores idosos


In [ ]:
lista_faixa_etaria_jovem = ['16 anos', '17 anos', '18 anos']
df_eleitores_trim_2020 = df_eleitores_2020.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_mais_jovens = df_eleitores_trim_2020.query(f'DS_FAIXA_ETARIA in {lista_faixa_etaria_jovem}').groupby(
    'NM_MUNICIPIO')[['NM_MUNICIPIO']].size().sort_values(ascending=False).head(3).reset_index(name='count')
fig = px.bar(df_mais_jovens, x='NM_MUNICIPIO', y='count',
             title='Cidades com eleitores mais jovens')
fig.show()

create_download_link(df_mais_jovens, 'Cidades com eleitores mais jovens', "df_mais_jovens.csv")

In [ ]:
lista_faixa_etaria_idoso = ['60 a 64 anos', '65 a 69 anos', '70 a 74 anos', '75 a 79 anos',
                            '80 a 84 anos', '85 a 89 anos', '90 a 94 anos', '95 a 99 anos', '100 anos ou mais']
df_mais_velhos = df_eleitores_trim_2020.query(f'DS_FAIXA_ETARIA in {lista_faixa_etaria_idoso}').groupby(
    'NM_MUNICIPIO')[['NM_MUNICIPIO']].size().sort_values(ascending=False).head(3).reset_index(name='count')
fig = px.bar(df_mais_velhos, x='NM_MUNICIPIO', y='count',
             title='Cidades com eleitores mais velhos')
fig.show()

create_download_link(df_mais_velhos, 'Cidades com eleitores mais velhos', "df_mais_velhos.csv")

In [ ]:
# 05 Jornalista quer comparar a renda média de eleitores do Vale do Paraíba com a de outras regiões do Estado


In [ ]:
cobertura_vanguarda = {
    "APARECIDA": "VALE DO PARAÍBA",
    "CAÇAPAVA": "VALE DO PARAÍBA",
    "CACHOEIRA PAULISTA": "VALE DO PARAÍBA",
    "CANAS": "VALE DO PARAÍBA",
    "CUNHA": "VALE DO PARAÍBA",
    "GUARATINGUETÁ": "VALE DO PARAÍBA",
    "IGARATÁ": "VALE DO PARAÍBA",
    "JACAREÍ": "VALE DO PARAÍBA",
    "JAMBEIRO": "VALE DO PARAÍBA",
    "LAGOINHA": "VALE DO PARAÍBA",
    "LORENA": "VALE DO PARAÍBA",
    "NATIVIDADE DA SERRA": "VALE DO PARAÍBA",
    "PARAIBUNA": "VALE DO PARAÍBA",
    "PINDAMONHANGABA": "VALE DO PARAÍBA",
    "PIQUETE": "VALE DO PARAÍBA",
    "POTIM": "VALE DO PARAÍBA",
    "REDENÇÃO DA SERRA": "VALE DO PARAÍBA",
    "ROSEIRA": "VALE DO PARAÍBA",
    "SANTA BRANCA": "VALE DO PARAÍBA",
    "SÃO JOSÉ DOS CAMPOS": "VALE DO PARAÍBA",
    "SÃO LUÍS DO PARAITÍNGA": "VALE DO PARAÍBA",
    "TAUBATÉ": "VALE DO PARAÍBA",
    "TREMEMBÉ": "VALE DO PARAÍBA",

    "ARAPEÍ": "VALE HISTÓRICO",
    "AREIAS": "VALE HISTÓRICO",
    "BANANAL": "VALE HISTÓRICO",
    "CRUZEIRO": "VALE HISTÓRICO",
    "LAVRINHAS": "VALE HISTÓRICO",
    "QUELUZ": "VALE HISTÓRICO",
    "SÃO JOSÉ DO BARREIRO": "VALE HISTÓRICO",
    "SILVEIRAS": "VALE HISTÓRICO",

    "CARAGUATATUBA": "LITORAL NORTE",
    "ILHABELA": "LITORAL NORTE",
    "SÃO SEBASTIÃO": "LITORAL NORTE",
    "UBATUBA": "LITORAL NORTE",

    "CAMPOS DO JORDÃO": "SERRA DA MANTIQUEIRA",
    "MONTEIRO LOBATO": "SERRA DA MANTIQUEIRA",
    "SANTO ANTÔNIO DO PINHAL": "SERRA DA MANTIQUEIRA",
    "SÃO BENTO DO SAPUCAÍ": "SERRA DA MANTIQUEIRA",
    
    "ATIBAIA": "REGIÃO BRAGANTINA",
    "BOM JESUS DOS PERDÕES": "REGIÃO BRAGANTINA",
    "BRAGANÇA PAULISTA": "REGIÃO BRAGANTINA",
    "JOANÓPOLIS": "REGIÃO BRAGANTINA",
    "NAZARÉ PAULISTA": "REGIÃO BRAGANTINA",
    "PIRACAIA": "REGIÃO BRAGANTINA",
    "VARGEM": "REGIÃO BRAGANTINA",
}

dfmc = pd.DataFrame(columns=["Localidade", "regiao"])
for cidade in cobertura_vanguarda.keys():
    dfmc = dfmc.append(
        pd.DataFrame({"Localidade": [cidade], "regiao": [
                     cobertura_vanguarda[cidade]]})
    )


df_salarios["Localidade"] = df_salarios["Localidade"].str.upper()
medias_salariais = (
    df_salarios.query('Nome == "Salário médio mensal"')[["Localidade", "2019"]]
    .set_index("Localidade")
    .join(dfmc.set_index("Localidade"), how="inner")
    .groupby("regiao")
    .mean()
    .reset_index("regiao")
)
fig = px.bar(medias_salariais, x="regiao", y="2019", title="Media salarial das regioes")
fig.show()

create_download_link(medias_salariais, "Media salarial das regioes", "media_salarial_regiao.csv")

In [ ]:
# 07 Jornalista quer poder relacionar características do eleitorado de acordo com o representante eleito em determinada região de eleições futuras

In [ ]:
arquivo_candidatos_2012 = './data/consulta_cand_2020_SP.csv'

df_candidatos_2020 = pd.read_csv(
    arquivo_candidatos_2020,
    encoding="iso-8859-1",
    error_bad_lines=False,
    sep=";"
)

candidato_sjc = df_candidatos_2020.query('DS_SIT_TOT_TURNO == "ELEITO" and DS_CARGO == "PREFEITO" and NM_UE == "SÃO JOSÉ DOS CAMPOS"')[
    ['NM_CANDIDATO', 'NM_UE', 'DS_GRAU_INSTRUCAO', 'DS_ESTADO_CIVIL']]

nome_prefeito = candidato_sjc.NM_CANDIDATO.values[0]
grau_instrucao = candidato_sjc.DS_GRAU_INSTRUCAO.values[0]
estado_civil = candidato_sjc.DS_ESTADO_CIVIL.values[0]

classificado_grau_escolaridade = df_eleitores_2020 \
    .query('NM_MUNICIPIO == "SÃO JOSÉ DOS CAMPOS"')['DS_GRAU_ESCOLARIDADE'] \
    .value_counts() \
    .rename_axis("grau_escolaridade") \
    .reset_index(name="counts")

fig = px.bar(
    classificado_grau_escolaridade,
    x="grau_escolaridade",
    y="counts",
    title=f"Grau escolaridade eleitorado de SÃO JOSÉ DOS CAMPOS em 2020 - {nome_prefeito}, {grau_instrucao}",
)

fig.show()

create_download_link(classificado_grau_escolaridade, f"Grau escolaridade eleitorado de SÃO JOSÉ DOS CAMPOS em 2020 - {nome_prefeito}, {grau_instrucao}", "classificado_grau_escolaridade.csv")

In [ ]:
classificado_estado_civil = df_eleitores_2020 \
    .query('NM_MUNICIPIO == "SÃO JOSÉ DOS CAMPOS"')['DS_ESTADO_CIVIL'] \
    .value_counts() \
    .rename_axis("estado_civil") \
    .reset_index(name="counts")

fig = px.bar(
    classificado_estado_civil,
    x="estado_civil",
    y="counts",
    title=f"Estado civil eleitorado de SÃO JOSÉ DOS CAMPOS em 2020 - {nome_prefeito}, {estado_civil}",
)

fig.show()

create_download_link(classificado_estado_civil, f"Estado civil eleitorado de SÃO JOSÉ DOS CAMPOS em 2020 - {nome_prefeito}, {estado_civil}", "classificado_estado_civil.csv")

In [ ]:
# 08 Jornalista quer saber a percentagem de eleitores ausentes da última eleição

In [ ]:
arquivo_comparecimento = './data/perfil_comparecimento_abstencao_2020.csv'

df_comparecimento = pd.read_csv(
    arquivo_comparecimento,
    encoding="iso-8859-1",
    error_bad_lines=False,
    sep=";"
)

soma_total = df_comparecimento[['QT_APTOS', 'QT_ABSTENCAO']].sum()
porcentagem_abstencao = soma_total.QT_ABSTENCAO / soma_total.QT_APTOS

print(f"A porcentagem de eleitores ausentes da última eleição foi: %.2f" %
      (porcentagem_abstencao * 100))

In [ ]:
# 09 Jornalista quer saber a evolução do número de eleitores


In [ ]:
classificacao_ano_eleicao = (
    df_eleitores_2020.append(df_eleitores_2016)
    ["ANO_ELEICAO"]
    .value_counts()
    .rename_axis("ano_eleicao")
    .reset_index(name="counts")
)


fig = px.bar(
    classificacao_ano_eleicao,
    x="ano_eleicao",
    y="counts",
    title="Número de eleitores por ano",
)

fig.show()

create_download_link(classificacao_ano_eleicao, "Número de eleitores por ano", "classificacao_ano_eleicao.csv")